In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
print(tf.__version__)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

1.5.0


In [22]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
t1, sr=librosa.load('test_x_01.wav', sr=None)
T1=librosa.stft(t1, n_fft=1024, hop_length=512)
t2, sr=librosa.load('test_x_02.wav', sr=None)
T2=librosa.stft(t2, n_fft=1024, hop_length=512)

In [24]:
print(S.shape, X.shape, T1.shape, T2.shape)

(513, 2459) (513, 2459) (513, 142) (513, 380)


In [25]:
Clean_mod = np.abs(S)
Dirty_mod = np.abs(X)
T1_mod = np.abs(T1)
T2_mod = np.abs(T2)

In [26]:
print(Clean_mod.shape, Dirty_mod.shape, T1_mod.shape, T2_mod.shape)

(513, 2459) (513, 2459) (513, 142) (513, 380)


In [27]:
Clean_Trans = np.transpose(Clean_mod)
Dirty_Trans = Dirty_mod.transpose()
T1_Dirty_Trans = T1_mod.transpose()
T2_Dirty_Trans = T2_mod.transpose()

In [28]:
print(Clean_Trans.shape, Dirty_Trans.shape, T1_Dirty_Trans.shape, T2_Dirty_Trans.shape)

(2459, 513) (2459, 513) (142, 513) (380, 513)


In [29]:
# Creating the Place Holders

X = tf.placeholder(tf.float32, shape = [None, 513])

Y_true = tf.placeholder(tf.float32, shape = [None, 513])

hold_prob = tf.placeholder(tf.float32)

In [47]:
# Defining the Hidden Layers

HL1 = 670

HL2 = 670

OL = 513

In [48]:
# Defining the weights and bias for the five layers

W1 = tf.Variable(tf.truncated_normal([513, HL1], stddev=0.1))
b1 = tf.Variable(tf.zeros([HL1]))

W2 = tf.Variable(tf.truncated_normal([HL1, HL2], stddev=0.1))
b2 = tf.Variable(tf.zeros([HL2]))

W3 = tf.Variable(tf.truncated_normal([HL2, OL], stddev=0.1))
b3 = tf.Variable(tf.zeros([OL]))

In [49]:
# Defining the activation function for each layer

L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1_dropout = tf.nn.dropout(L1, hold_prob)

L2 = tf.nn.relu(tf.matmul(L1_dropout, W2) + b2)
L2_dropout = tf.nn.dropout(L2, hold_prob)

Y_logits = tf.matmul(L2_dropout, W3) + b3

Y_pred = tf.nn.sigmoid(Y_logits)

In [50]:
# Defining the Loss Function

MSE = tf.reduce_mean(tf.pow(Y_true - Y_pred, 2))

In [51]:
# Defining the optimizer function

optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(MSE)

In [52]:
# Creating the Session

init = tf.global_variables_initializer()

In [53]:
# Creating a function that returns random batches of rows for training

def Batching(x, y, batch_size, shuffle = False):
    if shuffle:
        indices = np.arange(x.shape[0])
        np.random.shuffle(indices)
    for idx in range(0, x.shape[0] - batch_size + 1, batch_size):
        if shuffle:
            batch = indices[idx: idx + batch_size]
        else:
            batch = slice(idx, idx + batch_size)
        yield x[batch], y[batch]

In [54]:
epochs = 5000

sess = tf.Session(config = config)
sess.run(init)
    
for i in range(epochs + 1):

    for batch in Batching(Dirty_Trans, Clean_Trans, 128, shuffle = True):
         batch_X , batch_Y = batch
    
         sess.run(train, feed_dict={X: batch_X, Y_true: batch_Y, hold_prob : 0.7})

    if i% 1000 == 0:
        loss = sess.run(MSE, feed_dict={X: batch_X, Y_true: batch_Y, hold_prob : 0.7})

        #acc_tst, loss_tst = sess.run([accuracy, cross_entropy], feed_dict={X: mnist.test.images, Y_true: mnist.test.labels})

        print("Currently on Epoch # {}".format(i))
        print("Training Loss = {}".format(loss))
        #print("Test Set Accuracy and Loss: Accuracy = {}, Loss = {}".format(acc_tst*100, loss_tst*100))
        print("\n")

Currently on Epoch # 0
Training Loss = 0.076544851064682


Currently on Epoch # 1000
Training Loss = 0.05235903337597847


Currently on Epoch # 2000
Training Loss = 0.04080304875969887


Currently on Epoch # 3000
Training Loss = 0.04765787720680237


Currently on Epoch # 4000
Training Loss = 0.0432356521487236


Currently on Epoch # 5000
Training Loss = 0.04803098738193512




Converting the cleaned test signal back to audio signal

Test Signal 1

In [55]:
T1_Clean_Trans = sess.run(Y_pred, feed_dict={X: T1_Dirty_Trans, hold_prob : 0.7})
T1_Clean = np.transpose(T1_Clean_Trans)
Phase = np.divide(T1, T1_mod)
T1_Clean_Phase = np.multiply(T1_Clean,Phase)

In [56]:
sh_T1 = librosa.istft(T1_Clean_Phase,hop_length=512)
librosa.output.write_wav('test_s_recons_01.wav', sh_T1, sr)

Test Signal 2

In [57]:
T2_Clean_Trans = sess.run(Y_pred, feed_dict={X: T2_Dirty_Trans, hold_prob : 0.7})
T2_Clean = np.transpose(T2_Clean_Trans)
Phase = np.divide(T2, T2_mod)
T2_Clean_Phase = np.multiply(T2_Clean,Phase)

In [58]:
sh_T2 = librosa.istft(T2_Clean_Phase,hop_length=512)
librosa.output.write_wav('test_s_recons_02.wav', sh_T2, sr)